In [29]:
def foo():
    #raise Exception('an error inside foo')
    print('inside foo')

In [30]:
def deco(fun):
    def f():
        print('inside deco')
        return fun()
    return f

def neva_fail(fun):
    def f():
        try:
            return fun()
        except Exception as err:
            print(f'an err: {err}')
    return f

In [31]:
neva_fail(deco(foo))()

inside deco
inside foo


In [32]:
@neva_fail
@deco
def foo2():
    return foo()

In [33]:
foo2()

inside deco
inside foo


In [37]:
@neva_fail
deco(foo)

SyntaxError: invalid syntax (21070375.py, line 2)

In [1]:
# an example event queue command
event = "SlewTo(100.0,45.0)..with..a..XYOffset(5,5)..and..track()"

In [ ]:
def point_to_az_el(*args): # slew
    pass

def slew_to_az_el(*args): # slew and maintian az, el position
    pass

def halt_telescope(*args):
    pass

def offset_ra_dec(*args):
    pass

def offset_x_y(*args):
    pass



acqKeys = ("SlewTo", "XYOffset", ]
def translate_sub_event():

In [39]:
fStr = [x for x in event.split('..') if '(' in x]

for f in fStr:
    print(f)


SlewTo(100.0,45.0)
XYOffset(5,5)
track()


In [31]:
import time

def set_exposure_time_blue(exposure_time):
    print(f'[KCWI]Setting blue camera exposure time to {exposure_time}')

def load_configuration(grating_blue, cwave_blue, grating_red, cwave_red, slicer):
    print(f'[KCWI]Setting KCWI configuration to: {grating_blue} at {cwave_blue}, '
          f'{grating_red} at {cwave_red},'
          f'with slicer {slicer}')

def set_observation_type(type):
    if type == "T":
        observation_type = "Target"
    elif type == "S":
        observation_type = "Sky"
    else:
        observation_type = "Undefined"
    print(f'[KCWI] Setting KCWI observation type to {observation_type}')

def start_exposure_blue():
    print(f'[KCWI] Checking telescope')
    print(f'[KCWI] Checking guider')
    print(f'[KCWI] Checking instrument')
    print(f'[KCWI] Exposing on blue camera')
    time.sleep(1)
    return "raw_file"

def offset(ra_offset, dec_offset):
    print(f'[K2] Offsetting K2 telescope by {ra_offset}, {dec_offset}')
    print(f'[K2] Waiting for telescope to settle...')
    time.sleep(1)
    
def guiding_mode(mode):
    print(f'[Guider] Setting guiding mode to {mode}')

def start_guiding():
    print(f'[Guider] Start guiding')
    print(f'[Guider] Wait for guiding...')
    time.sleep(1)
    
def send_file_to_drp(filename):
    print(f'[DRP] Sending {filename} to DRP server')
    print(f'[DRP] Trigger <ingest_file> on KCWI_DRP')
    time.sleep(1)

In [32]:
class ObservingScript():
    
    def __init__(self):
        self.name = None
        self.version = None
        self.observation = None
        self.observation_parameters = None
    
    def loadMetadata(self):
        self.metadata = self.observation['metadata']
        self.name = self.metadata['name']
        self.version = self.metadata['version']
        
    def loadParameters(self):
        self.observation_parameters = self.observation['observation_parameters']
        
    def loadObservation(self):
        self.loadMetadata()
        self.loadParameters()
        
    def _pre_condition(self):
        return True

    def _post_condition(self):
        return True

    def _get_value(self, key):
        try:
            return self.observation_parameters[key]
        except KeyError:
            return "Undefined"
    
    def diagnostic(self):
        for key in list(self.observation_parameters.keys()):
            print("%s -> %s" % (key, str(self._get_value(key))))
    
    def _perform(self):
        raise Exception("Not yet implemented")

    def apply(self):
        self.loadObservation()
        if self._pre_condition():
            self.output = self._perform()
            if self._post_condition():
                return self.output
        return None

    def __call__(self):
        """
        Makes objects of this calls callable.
        """
        return self.apply()
    
class KCWI_ifu_sci_dither_script(ObservingScript):
    
    def __init__(self):
        ObservingScript.__init__(self)
        self.template = None
        
    def _perform(self):
        set_exposure_time_blue(self._get_value('DET1.EXPTIME'))
        load_configuration(
                grating_blue = self._get_value('CFG.CAM1.GRATING'), 
                cwave_blue = self._get_value('CFG.CAM1.CWAVE'),
                grating_red = self._get_value('CFG.CAM2.GRATING'),
                cwave_red = self._get_value('CFG.CAM2.CWAVE'),
                slicer = self._get_value('CFG.SLICER'))
        dither_counter = 1
        number_of_positions = self._get_value('SEQ.NDITHER')
        for dither in self._get_value('SEQ.DITARRAY'):
            print(f'\n--- Offset position {dither_counter} of {number_of_positions}')
            offset(dither[0], dither[1])
            set_observation_type(dither[2])
            guiding_mode(dither[3])
            if dither[3] == 'Guided':
                start_guiding()
            filename = start_exposure_blue()
            send_file_to_drp(filename)
            dither_counter += 1

In [ ]:
class KCWI_ifu_sci_dither_script(ObservingScript):
    
    def __init__(self):
        ObservingScript.__init__(self)
        self.template = None
        
    def _perform(self):
        set_exposure_time_blue(self._get_value('DET1.EXPTIME'))
        load_configuration(
                grating_blue = self._get_value('CFG.CAM1.GRATING'), 
                cwave_blue = self._get_value('CFG.CAM1.CWAVE'),
                grating_red = self._get_value('CFG.CAM2.GRATING'),
                cwave_red = self._get_value('CFG.CAM2.CWAVE'),
                slicer = self._get_value('CFG.SLICER'))
        dither_counter = 1
        number_of_positions = self._get_value('SEQ.NDITHER')
        for dither in self._get_value('SEQ.DITARRAY'):
            print(f'\n--- Offset position {dither_counter} of {number_of_positions}')
            offset(dither[0], dither[1])
            set_observation_type(dither[2])
            guiding_mode(dither[3])
            if dither[3] == 'Guided':
                start_guiding()
            filename = start_exposure_blue()
            send_file_to_drp(filename)
            dither_counter += 1

In [33]:
science_observation = {
    "metadata": {
        "name": "KCWI_ifu_sci_dither",
        "version": "0.1"
    },
    "observation_parameters": {
        "DET1.EXPTIME": 60,
        "DET1.NEXP": 2,
        "DET2.EXPTIME": 60,
        "DET2.NEXT": 2,
        "CFG.CAM1.GRATING": "BM",
        "CFG.CAM1.CWAVE": 4500,
        "CFG.SLICER": "Medium",
        "SEQ.NDITHER": 4,
        "SEQ.DITARRAY": [
            [0,0,"T","Guided"],
            [5,5,"T","Guided"],
            [-10,-10, "T", "Guided"],
            [30,30,"S","Unguided"]
            ]
    }
}


In [34]:
myscript = KCWI_ifu_sci_dither_script()
myscript.observation = science_observation
myscript.loadObservation()

In [35]:
myscript.diagnostic()

myscript.apply()



DET1.EXPTIME -> 60
DET1.NEXP -> 2
DET2.EXPTIME -> 60
DET2.NEXT -> 2
CFG.CAM1.GRATING -> BM
CFG.CAM1.CWAVE -> 4500
CFG.SLICER -> Medium
SEQ.NDITHER -> 4
SEQ.DITARRAY -> [[0, 0, 'T', 'Guided'], [5, 5, 'T', 'Guided'], [-10, -10, 'T', 'Guided'], [30, 30, 'S', 'Unguided']]
[KCWI]Setting blue camera exposure time to 60
[KCWI]Setting KCWI configuration to: BM at 4500, Undefined at Undefined,with slicer Medium

--- Offset position 1 of 4
[K2] Offsetting K2 telescope by 0, 0
[K2] Waiting for telescope to settle...
[KCWI] Setting KCWI observation type to Target
[Guider] Setting guiding mode to Guided
[Guider] Start guiding
[Guider] Wait for guiding...
[KCWI] Checking telescope
[KCWI] Checking guider
[KCWI] Checking instrument
[KCWI] Exposing on blue camera
[DRP] Sending raw_file to DRP server
[DRP] Trigger <ingest_file> on KCWI_DRP

--- Offset position 2 of 4
[K2] Offsetting K2 telescope by 5, 5
[K2] Waiting for telescope to settle...
[KCWI] Setting KCWI observation type to Target
[Guider] Setti